# Working

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("src/")

In [2]:

from src.data import get_track ,padBatch, getBatch, preprocessBatch, fourier_mapping
from IPython.display import clear_output
import tensorflow as tf 
import numpy as np
import math

In [3]:
trk = get_track()
clear_output()

# Experiment

In [4]:
batch = getBatch(trk,batch_size=32)

In [9]:
X = batch[:,:,2:5]


In [6]:
from src.model import RandomFourierFeatures

In [14]:
lx = RandomFourierFeatures()
xpos = lx(X[:,:,0])
ypos = lx(X[:,:,1])
dE =   lx(X[:,:,2])

In [15]:
XS = tf.stack([xpos,ypos,dE],axis=2)

In [17]:
lidx = 1

In [22]:
xc = XS[:,-lidx]
xp = XS[:,-lidx-1]
x = tf.concat([xc,xp],axis=2)


<tf.Tensor: shape=(32, 3, 32), dtype=float32, numpy=
array([[[0.95198643, 0.9725376 , 0.8837802 , ..., 0.9574801 ,
         0.95295703, 0.7307978 ],
        [0.8492578 , 0.96280503, 0.92901194, ..., 0.9226678 ,
         0.9995295 , 0.97431433],
        [0.99954635, 0.9636922 , 0.722705  , ..., 0.90289855,
         0.85081893, 0.7078998 ]],

       [[0.95198643, 0.9725376 , 0.8837802 , ..., 0.9574801 ,
         0.95295703, 0.7307978 ],
        [0.8492578 , 0.96280503, 0.92901194, ..., 0.9226678 ,
         0.9995295 , 0.97431433],
        [0.99954635, 0.9636922 , 0.722705  , ..., 0.90289855,
         0.85081893, 0.7078998 ]],

       [[0.95198643, 0.9725376 , 0.8837802 , ..., 0.9574801 ,
         0.95295703, 0.7307978 ],
        [0.8492578 , 0.96280503, 0.92901194, ..., 0.9226678 ,
         0.9995295 , 0.97431433],
        [0.99954635, 0.9636922 , 0.722705  , ..., 0.90289855,
         0.85081893, 0.7078998 ]],

       ...,

       [[0.95198643, 0.9725376 , 0.8837802 , ..., 0.9574801 ,
  